In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
from sklearn.metrics import precision_score, f1_score, recall_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device object

In [2]:
test_dir = "/Users/geral/OneDrive/Desktop/NUS/Y4S2/BT4222/Project/bt4222_project/test/"
test_hostile_dir = "/Users/geral/OneDrive/Desktop/NUS/Y4S2/BT4222/Project/bt4222_project/test/hostile_images/"
test_non_hostile_dir = "/Users/geral/OneDrive/Desktop/NUS/Y4S2/BT4222/Project/bt4222_project/test/non_hostile_images/"

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [4]:
test_dataset = datasets.ImageFolder(test_dir, transform)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=True, num_workers=8)

In [8]:
print('Test dataset size:', len(test_dataset))
class_names = test_dataset.classes
print('Class names:', class_names)

Test dataset size: 682
Class names: ['hostile_images', 'non_hostile_images']


In [6]:
model = models.resnet18(pretrained=True)   #load resnet18 model
num_features = model.fc.in_features #extract fc layers features
model.fc = nn.Linear(num_features, 2)#(num_of_class == 2)
model.load_state_dict(torch.load("model_resnet18.pth"))
model.to(device)

C:\Users\geral\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\geral\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 80
plt.rcParams.update({'font.size': 20})
def imshow(input, title):
    # torch.Tensor => numpy
    input = input.numpy().transpose((1, 2, 0))
    # undo image normalization
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    input = std * input + mean
    input = np.clip(input, 0, 1)
    # display images
    plt.imshow(input)
    plt.title(title)
    plt.show()
##Testing
model.eval()
start_time = time.time()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
with torch.no_grad():
    running_loss = 0.
    running_corrects = 0
    for i, (inputs, labels) in enumerate(test_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        if i == 0:
            print('======>RESULTS<======')
            images = torchvision.utils.make_grid(inputs[:])
            #imshow(images.cpu(),title="results")
            results = pd.DataFrame(index=range(1,len(test_dataset)+1), data={"pred": [class_names[x] for x in preds[:]], "actual": [class_names[x] for x in labels[:]]})
    epoch_loss = running_loss / len(test_dataset)
    epoch_acc = running_corrects/ len(test_dataset)
    epoch_prec = precision_score(labels.data, preds)
    epoch_f1 = f1_score(labels.data, preds)
    epoch_recall = recall_score(labels.data, preds)
    print('[Test #{}] Loss: {:.5f} Acc: {:.5f} Prec: {:.5f} F1-score: {:.5f} Recall: {:.5f} Time: {:.5f}s'.
          format(1, epoch_loss, epoch_acc, epoch_prec, epoch_f1, epoch_recall, time.time() - start_time))

======>RESULTS<======
[Test #1] Loss: 0.00914 Acc: 0.99560 Prec: 0.99840 F1-score: 0.99760 Recall: 0.99681 Time: 26.89903s
